# import

In [4]:

import awswrangler as wr
import pandas as pd
# import pymysql
# import datetime

    

# Lambda 핸들러 함수. 이벤트와 컨텍스트 매개변수를 받습니다.


# 첫 번째 SQL 쿼리 실행
sql_query_all = """
with raw1 as (
select
id
, status
,reg_ts
, mvno
, order_phone_number
, customer_phone_number
, customer_name
, plan_name
, completed_at
, case when completed_at is not null then 1 else 0 end as is_completed
, case when completed_at between  date( '2023-06-01') and date('2023-06-30') then 1 else 0 end as is_completed_june
from moyo_order.mobile_plan_order a
where 1=1

    and a.mvno = '미니게이트'
    and a.reg_ts >= date( '2023-05-01')
    and a.reg_ts < date( '2023-07-25')
)
select *
from raw1

"""

df = wr.athena.read_sql_query(sql=sql_query_all, database="moyo_logs")

In [11]:
df_mini = pd.read_csv('2306 수수료 지급액_모요.xlsx - 2306_rawdata.csv')

In [14]:
df_mini.head()

,신청일,통신망,가입유형,가입자 번호,개통일,가입자,요금제 명,요금제 가격,상태,모요 수수료\r\n(VAT 별도)
0,2023.05.27 21:57:48,KT,번호이동,1029071914,2023.06.02,엄재환,미니 LTE 11GB+,-,개통완료,"5,000"
1,2023.05.28 01:00:32,KT,번호이동,1099508804,2023.06.02,최어진,미니 LTE 11GB+,-,개통완료,"5,000"
2,2023.05.28 09:23:28,KT,신규가입,1048826446,2023.06.02,이경환,미니 LTE 11GB+,-,개통완료,"5,000"
3,2023.05.28 15:05:37,KT,신규가입,1059450346,2023.06.02,김무엽,미니 LTE 11GB+,-,개통완료,"5,000"
4,2023.05.28 16:07:58,KT,번호이동,1083041030,2023.06.02,나형돈,미니 LTE 7GB+,-,개통완료,"5,000"


In [13]:
df_mini.columns

Index(['신청일', '통신망', '가입유형', '가입자 번호', '개통일', '가입자', '요금제 명', ' 요금제 가격', '상태',
       ' 모요 수수료\r\n(VAT 별도)'],
      dtype='object')

In [9]:
df.head(2)

,id,status,reg_ts,mvno,order_phone_number,customer_phone_number,customer_name,plan_name,completed_at,is_completed,is_completed_june
0,196940,ACTIVATION_COMPLETE,2023-06-02 00:06:57,미니게이트,01033911724,01033911724,정다은,미니 LTE 11GB+,2023-06-30 12:16:18,1,0
1,196945,ACTIVATION_COMPLETE,2023-06-02 00:23:12,미니게이트,01030243404,01030243404,이원준,미니 LTE 11GB+,2023-06-05 15:33:00,1,1


# EDA

In [6]:
df[df['is_completed_june']==1].count()

id                       5516
status                   5516
reg_ts                   5516
mvno                     5516
order_phone_number       5516
customer_phone_number    5516
customer_name            5516
plan_name                5516
completed_at             5516
is_completed             5516
is_completed_june        5516
dtype: int64

# 전처리

In [15]:
df_mini['가입자 번호'] = '0'+df_mini['가입자 번호'].astype(str)

In [16]:
df_mini['가입자 번호'].head()

0    01029071914
1    01099508804
2    01048826446
3    01059450346
4    01083041030
Name: 가입자 번호, dtype: object

# merge

In [20]:
result = pd.merge(df, df_mini, how='outer', left_on='customer_phone_number', right_on='가입자 번호')

In [21]:
result.head()

,id,status,reg_ts,mvno,order_phone_number,customer_phone_number,customer_name,plan_name,completed_at,is_completed,...,신청일,통신망,가입유형,가입자 번호,개통일,가입자,요금제 명,요금제 가격,상태,모요 수수료\r\n(VAT 별도)
0,196940,ACTIVATION_COMPLETE,2023-06-02 00:06:57,미니게이트,01033911724,01033911724,정다은,미니 LTE 11GB+,2023-06-30 12:16:18,1,...,2023.06.02 00:06:57,KT,번호이동,01033911724,2023.06.30,정다은,미니 LTE 11GB+,-,개통완료,"5,000"
1,196945,ACTIVATION_COMPLETE,2023-06-02 00:23:12,미니게이트,01030243404,01030243404,이원준,미니 LTE 11GB+,2023-06-05 15:33:00,1,...,2023.06.02 00:23:12,KT,번호이동,01030243404,2023.06.05,이원준,미니 LTE 11GB+,-,개통완료,"5,000"
2,196946,ACTIVATION_COMPLETE,2023-06-02 00:24:13,미니게이트,01086294354,01086294354,이재호,미니 LTE 11GB+,2023-06-08 13:05:58,1,...,2023.06.02 00:24:13,KT,번호이동,01086294354,2023.06.08,이재호,미니 LTE 11GB+,-,개통완료,"5,000"
3,196948,ACTIVATION_COMPLETE,2023-06-02 00:27:28,미니게이트,01074256516,01038936516,김경용,미니 LTE 11GB+,2023-06-02 19:30:18,1,...,2023.06.02 00:27:28,KT,신규가입,01038936516,2023.06.02,김경용,미니 LTE 11GB+,-,개통완료,"5,000"
4,196973,APPLICATION_CANCELED,2023-06-02 01:10:36,미니게이트,01033345934,01033345934,강인웅,미니 LTE 11GB+,NaT,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
result.to_csv('2306_mini.csv', index=False)